# Convert OpenSN Host Galaxies to host,x,y,z

Data downloaded from The Open Supernova Catalog https://sne.space on Dec. 2, 2019

In [0]:
import pandas as pd
import numpy as np
from astropy import units
from astropy.coordinates import SkyCoord, Distance
from astropy.cosmology import WMAP9
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
df = pd.read_csv('Host Galaxies – The Open Supernova Catalog.csv')
# Select Host Galaxy information that has complete position data
df = df.loc[(df['Galaxy/Cluster'].notnull()) & \
              (df['Host R.A.'].notnull()) & \
              (df['Host Dec.'].notnull()) & \
              (df['Host dL (Mpc)'].notnull()) & \
              (df['Host z'].notnull())]
print(len(df))
df

8272


,Galaxy/Cluster,Host R.A.,Host Dec.,Host z,Host dL (Mpc)
1,NGC 4419,12:26:57,+15:02:52,-0.00092,17
3,IC 3476,12:32:42,+14:03:04,-0.00057,12
4,IC 3311,12:25:33,+12:15:36,-0.00055,17
7,"ESO 56-G115, LMC",05:23:35,-69:45:22,1.13e-05,0.043
9,NGC 6946,20:34:52,+60:09:15,0.00015,4.7
...,...,...,...,...,...
10701,CXO J123709.4+622214,12:37:09.4,+62:22:14,1.713*,13182*
10702,"A123644+6212, GOODS J123644.13+621245.2",12:36:44,+62:12:44,1.755,13580*
10703,[KPA2012] GN_IRS18,12:37:16.823,+62:16:42.65,1.83*,14290*
10704,"A134709+0223, SCP J134709.90+022357.5",13:47:09,+02:23:57,2.07*,16620*


In [34]:
x = []
y = []
z = []

name = []

for index, row in df.iterrows():
    badAngle = False

    
    ra = row['Host R.A.']
    pos = str(ra).find(',')
    if (pos != -1):    
        ra = row['Host R.A.'][0:pos]

    dec = row['Host Dec.']
    pos = str(dec).find(',')
    if (pos != -1):    
        dec = row['Host Dec.'][0:pos]

    # Check to make sure angles are formatted correctly
    pos = str(ra).find(':') # Case: "-999"
    if(pos == -1):
      badAngle = True
    else:
      a = int(ra[0:pos])  
      # Case: "-99.00:...:..."
      if(a < -90 or a > 90):
        badAngle = True;
    pos = str(dec).find(':')
    if(pos == -1):
      badAngle = True
    else:
      a = int(dec[0:pos])
      if(a < -90 or a > 90):
        badAngle = True;

    if (not badAngle):
        # Convert dL (Mpc) to z
        # for some odd reason, z and dL (Mpc)read as str, not float (like in SNe set)
        pos = str(row['Host z']).find('*')
        if(pos != -1):
          d = float(row['Host z'][0:pos])
        else:
          d = float(row['Host z'])

        # Convert z to  comoving_distance
        c1 = SkyCoord(ra, dec, unit=(units.hourangle, units.deg), distance=WMAP9.comoving_distance(d)).galactic.cartesian
        x.append(c1.x.to(units.Mpc).value)
        y.append(c1.y.to(units.Mpc).value)
        z.append(c1.z.to(units.Mpc).value)


        name.append(row['Galaxy/Cluster'])
        

print("done")

done


In [35]:
distances = np.sqrt(np.array(x)**2. + np.array(y)**2. + np.array(z)**2.)
print(max(x))
print(max(y))
print(max(z))
print(max(distances))

2276.2848052050235
2862.713077229502
4758.501614321233
5630.105743543416


### Write this to a new csv file

In [37]:
print(len(name), len(x), len(y), len(z))
data = {'name':np.array(name),
        'x':np.array(x),
        'y':np.array(y),
        'z':np.array(z)}
pd.DataFrame(data).to_csv('OpenSNCatHostsConverted.csv', index=False)

8270 8270 8270 8270
